In [1]:
import os
import sys
sys.path.append("/mnt/d/aspiranture/code/sat_image_segmentation")
import dataset_tool.dataset6_generator

In [2]:
import importlib

In [3]:
import dataset_6_binary_tree
importlib.reload(dataset_6_binary_tree)

2023-10-01 21:23:46.290641: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-01 21:23:47.743320: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<module 'dataset_6_binary_tree' from '/mnt/d/aspiranture/code/sat_image_segmentation/dataset_6_binary_tree.py'>

In [47]:
categories_tree_1 = ["city",["forest",["water", "fields"]]]

In [48]:
ds = dataset_6_binary_tree.Dataset6BinaryTree(categories_tree_1)

In [49]:
ds.initialize()

Error read []
len 27


In [38]:
ds.get_supported_categories()

['city', 'forest', 'water', 'fields']

In [39]:
ds.categories_map

{'water': 1, 'forest': 2, 'fields': 3, 'city': 4, 'sands': 5, 'mountains': 6}

In [40]:
sup = [ds.categories_map[key] for key in ds.supported_categories]

In [41]:
ds.categories_map

{'water': 1, 'forest': 2, 'fields': 3, 'city': 4, 'sands': 5, 'mountains': 6}

In [50]:
ds.create_models()

Entering into iteration 1
Iteration 1 creating new arrays
Iteration 1 calcualte weights
Entering into iteration 2
Iteration 2 creating new arrays
Iteration 2 calcualte weights
Entering into iteration 3
Iteration 3 creating new arrays
Iteration 3 calcualte weights


In [28]:
type(ds.current_tree) == dataset_tool.btree.BTree

True

In [29]:
ds.models

In [30]:
predition_1 = ds.models[0].predict(ds.x_test_new, batch_size=2048)

792/792 [==============================] - 2s 3ms/step


In [119]:
y_tst_copy = ds.y_test_new.copy()

In [120]:
y_tst_copy[y_tst_copy == 1] = 0
y_tst_copy[y_tst_copy != 0] = 1


In [52]:
import tensorflow as tf

In [122]:
predition_1_vec = tf.argmax(predition_1, axis=-1).numpy()

In [80]:
from sklearn.metrics import accuracy_score

In [124]:
accuracy_score(y_tst_copy, predition_1_vec)

0.9940177622173092

In [51]:
ds.validate_with_test_data()

792/792 [==============================] - 2s 2ms/step
Binary acccucary for step 1 is 99.4%.
Calcaluting accuracy for 2 categories
Adding idxes for category 1: [      0       1       4 ... 1620628 1620629 1620630]
Total acccucary for step 1 is 99.4%.
306/306 [==============================] - 1s 3ms/step
Binary acccucary for step 2 is 97.16%.
Calcaluting accuracy for 3 categories
Adding idxes for category 2: [      7      42      44 ... 1620589 1620595 1620625]
Total acccucary for step 2 is 98.34%.
194/194 [==============================] - 1s 3ms/step
Binary acccucary for step 3 is 81.78%.
Calcaluting accuracy for 4 categories
Adding idxes for category 3: [      2       3       9 ... 1620612 1620622 1620627]
Total acccucary for step 3 is 93.97999999999999%.


{'water': 1, 'forest': 2, 'fields': 3, 'city': 4, 'sands': 5, 'mountains': 6}

In [207]:
for i,v in ds.categories_map.items():
    print(i)
    print(v)

water
1
forest
2
fields
3
city
4
sands
5
mountains
6


In [70]:
from tensorflow.keras.layers import *
import layers.naive_bayes
importlib.reload(layers.naive_bayes)
# from keras.models import Model, save_model
# from keras.layers import Input

# Create the model
# model_new_1 = tf.keras.models.Sequential()
# model_new_1.add(tf.keras.models.InputLayer(shape=(12,)))  # Input layer
# model_new_1.add(layers.naive_bayes.MultiClassNaiveBayesLayer(num_features=12, num_of_categories=7))  # Naive Bayes layer
# model_new_1.add(tf.keras.models.Softmax())  #

model_new_gnb = tf.keras.models.Sequential([
    layers.naive_bayes.MultiClassNaiveBayesLayer(num_features=12, num_of_categories=4, input_shape=(12,)),
    Activation("softmax")
    ])


In [71]:
model_new_gnb.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse'])
model_new_gnb.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_class_naive_bayes_la  (None, 4)                 96        
 yer_17 (MultiClassNaiveBay                                      
 esLayer)                                                        
                                                                 
 activation_17 (Activation)  (None, 4)                 0         
                                                                 
Total params: 96 (384.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 96 (384.00 Byte)
_________________________________________________________________


In [72]:
import numpy as np

In [73]:
y_int_labels = ds.y_train_new.copy()
y_int_labels = y_int_labels-1
means = []
variances = []

num_classes = 4  # or however many you have

for i in range(num_classes):
    means.append(np.mean(ds.x_train_new[y_int_labels == i], axis=0))
    variances.append(np.var(ds.x_train_new[y_int_labels == i], axis=0))

means = np.array(means)
variances = np.array(variances)

In [74]:
model_new_gnb.layers[0].set_weights([means, variances])

In [75]:
pred_res = model_new_gnb.predict(ds.x_test_new, batch_size=2048)

792/792 [==============================] - 3s 3ms/step


In [76]:
pred_res_vec = tf.argmax(pred_res, axis=-1).numpy()

In [77]:
pred_res_vec.max()

3

In [78]:
ds.y_test_new.max()

4

In [79]:
pred_res_vec = pred_res_vec + 1

In [81]:
accuracy_score(pred_res_vec, ds.y_test_new)

0.8963996122497966